Things to know before -

This is Logistic Regression from scratch. This is done and implemented in a way that this can easily be extended to Neural Networks.

For now, we can assume this to be a single Neuron with one output.

Formulas -

Compact form -

z = w.T X + b -> a = sigmoid(z) -> Loss l(a,y) = - [y * log(a) + (1-y) * log(1-a)]

Our goal - Find optimal w* and b* that minimizes the Loss.

Disclamer- We can write the whole formula in one line,
loss(w,b) =  -[ y * log(sigmoid(w.T X + b)) + (1-y) - log(1-sigmoid(w.T X+ b))]
We can minimize this also, but it gets a bit messy.

We are going to use the compact form to keep everything nice and clean.

Our goal is to get dl/dz. We can achieve that by finding dl/da * da/dz (chain rule).

dl/da= (a-y)/ a(1-a)

da/dz= a * (1-a)

=> dl/dz = a-y

Now, to update w after each iteration, we need to find dl/dw.

dl/dw = dl/dz * dz/dw

dz/dw = X

=> dl/dw= (a-y)*X

similarly update for b- 

dl/db = dl/dz*dz/db

dz/db= 1 

dl/db = a-y



***********************

**final update for all w's, w= 1/n * learning_rate * (a-y) * X**

**final update for all b's, w= 1/n * learning_rate * (a-y)**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.metrics as m
from sklearn.linear_model import LogisticRegression

In [ ]:
def generate_data():
  X,y=make_classification(n_samples=1000,n_features=10,random_state=42) # Generating random data
  X=np.concatenate((X,np.ones(X.shape[0]).reshape(X.shape[0],1) ), axis=1) # Adding a column of 1's to account for the bias term.
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
  lr=0.001
  lamb=0.01
  y_train=y_train.reshape(y_train.shape[0],1)
  y_test=y_test.reshape(y_test.shape[0],1)
  return X_train.T, X_test.T, y_train.T, y_test.T,lr,lamb

We have 1000 samples with each sample having 10 features.

Adding 1 to features to account for the bias term.

In [ ]:
def sigmoid(z):
  a=1 / ( 1 + np.exp(-(z) ) ) 
  return a

In [ ]:
def forward_prop(w,X):
  return sigmoid( w.T.dot(X) )

In [ ]:
def back_prop(X,a,y):
  dw = np.dot(X,(a-y).T)
  return dw

In [ ]:
def initialize_params(X):
  w=np.random.randn(X.shape[0],1)
  return w

In [ ]:
def loss(y, y_hat):
    loss = np.mean(-y*(np.log(y_hat)) + (1-y)*np.log(1-y_hat)) + np.mean(lamb/(2) * (w[0:10]**2))
    return loss

In [ ]:
def predict(w,X):
  return sigmoid( w.T.dot(X) )

In [ ]:
if __name__=='__main__':
  X_train, X_test, y_train, y_test,lr,lamb=generate_data()
  w=initialize_params(X_train)
  lo=[]
  accuracy=[]
  for i in range(0,500):
    predictions=[]
    a=forward_prop(w,X_train)
    dw=back_prop(X_train,a,y_train)
    w[0:10]= ((w[0:10] - ( lr * (dw[0:10]) )   ) + lamb* w[0:10] )/X_train.shape[1]
    w[10:11]=((w[10:11] - ( lr * (dw[10:11]) )   ) )/X_train.shape[1]
    for i in a.T:
      predictions.append(1 if i>=0.5 else 0)
    lo.append(loss(y_train,a))
    accuracy.append(m.accuracy_score(y_train.T,predictions))

In [ ]:
a=predict(w,X_test)
predictions_test=[]
for i in a.T:
  predictions_test.append(1 if i>=0.5 else 0)
print('acc on test set',m.accuracy_score(y_test.T,predictions_test))

acc on test set 0.8454545454545455


In [ ]:
clf=LogisticRegression()
clf.fit(X_train.T,y_train.T)
preds=clf.predict(X_test.T)
test=y_test.reshape(y_test.shape[1],)
print('sklearn accuracy on test',m.accuracy_score(test,preds))

sklearn accuracy on test 0.8454545454545455


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
